In [1]:
"""Data taken from https://www.arcgis.com/home/item.html?id=ae25571c60d94ce5b7fcbf74e27c00e0
Disclaimer: © Bundesamt für Kartographie und Geodäsie, Frankfurt am Main, 2011"""

import geopandas as gpd
import pandas as pd
import os
from shapely.geometry import Point

GER_SHAPE_PATH = os.path.join('..', 'data', 'external', 'vg2500_bld.shp')
GAS_STATIONS_PATH = os.path.join('..', 'data', 'raw', 'input_data', 'Eingabedaten', 'Tankstellen.csv')
gas_stations_df = pd.read_csv(GAS_STATIONS_PATH, sep=';', names=['id', 'Name', 'Company', 'Street', 'House_Number', 'Postalcode', 'City', 'Lat', 'Long'])
gas_stations_df.head()

,id,Name,Company,Street,House_Number,Postalcode,City,Lat,Long
0,1,star Tankstelle,STAR,Listlandstrasse,16,25992,List,55.015890,8.429530
1,2,Esso Tankstelle,ESSO,TRIFT 2,,25980,WESTERLAND,54.907468,8.308741
2,3,"SYLT, TRIFT",Shell,TRIFT 7,NaN,25980,SYLT,54.904400,8.310700
3,4,DEXX-Station 188,DEXX,Dorfstr.,19,25927,Aventoft,54.901300,8.822810
4,5,team Tankautomat Hesbüll,Raiffeisen,Raiffeisenstr.,1,25927,Hesbüll,54.900900,8.810690


In [2]:
ger_shape = gpd.read_file(GER_SHAPE_PATH)
ger_shape

,USE,RS,RS_ALT,GEN,SHAPE_LENG,SHAPE_AREA,geometry
0,2,02,020000000000,Hamburg,1.752538e+05,7.605398e+08,(POLYGON ((10.19113690277905 53.72299476403796...
1,2,03,030000000000,Niedersachsen,2.016496e+06,4.771641e+10,(POLYGON ((8.689616610395177 53.87999272032229...
2,2,04,040000000000,Bremen,1.549716e+05,4.054809e+08,"(POLYGON ((8.52975665105563 53.21639650603829,..."
3,2,05,050000000000,Nordrhein-Westfalen,1.352108e+06,3.404727e+10,"POLYGON ((8.666728266554577 52.52148958107465,..."
4,2,06,060000000000,Hessen,1.105093e+06,2.109232e+10,"POLYGON ((9.505598958397917 51.62776574465611,..."
5,2,07,070000000000,Rheinland-Pfalz,9.106504e+05,1.983491e+10,"POLYGON ((7.846572241934178 50.87860563659346,..."
6,2,08,080000000000,Baden-Württemberg,1.298892e+06,3.580140e+10,(POLYGON ((9.646340526898234 49.77781980827938...
7,2,09,090000000000,Bayern,2.046039e+06,7.055007e+10,"POLYGON ((10.47206532594119 47.5696445543698, ..."
8,2,10,100000000000,Saarland,2.829108e+05,2.578542e+09,"POLYGON ((7.068525902716757 49.62349163838737,..."
9,2,11,110000000000,Berlin,1.807512e+05,8.864801e+08,"POLYGON ((13.61191510028114 52.54331644291671,..."


In [4]:
# Pick the third gas station which is located in Sylt (Schleswig-Holstein)
def get_state_from_long_lat(long, lat):
    gas_station_point = Point(long, lat)
    for state_polygon in ger_shape.values:
        _, _, _, name, _, _, polygon = state_polygon
        if polygon.contains(gas_station_point):
            return name
    return 'unknown'

gas_stations_df['State'] = gas_stations_df.apply(lambda row: get_state_from_long_lat(row.Long, row.Lat), axis=1)
gas_stations_df.head()

,id,Name,Company,Street,House_Number,Postalcode,City,Lat,Long,State
0,1,star Tankstelle,STAR,Listlandstrasse,16,25992,List,55.015890,8.429530,Schleswig-Holstein
1,2,Esso Tankstelle,ESSO,TRIFT 2,,25980,WESTERLAND,54.907468,8.308741,Schleswig-Holstein
2,3,"SYLT, TRIFT",Shell,TRIFT 7,NaN,25980,SYLT,54.904400,8.310700,Schleswig-Holstein
3,4,DEXX-Station 188,DEXX,Dorfstr.,19,25927,Aventoft,54.901300,8.822810,Schleswig-Holstein
4,5,team Tankautomat Hesbüll,Raiffeisen,Raiffeisenstr.,1,25927,Hesbüll,54.900900,8.810690,Schleswig-Holstein


In [6]:
abbreviations = {'Baden-Württemberg': 'BW', 'Bayern': 'BY', 'Berlin': 'BE', 'Brandenburg': 'BB', 'Bremen': 'HB', 'Hamburg': 'HH', 'Hessen': 'HE', 'Mecklenburg-Vorpommern': 'MV', 'Niedersachsen': 'NI', 'Nordrhein-Westfalen': 'NW', 'Rheinland-Pfalz': 'RP', 'Saarland': 'SL', 'Sachsen': 'SN', 'Sachsen-Anhalt': 'ST', 'Schleswig-Holstein': 'SH', 'Thüringen': 'TH', 'unknown': 'unknown'}
gas_stations_df['State'] = gas_stations_df['State'].apply(lambda x: abbreviations[x])
gas_stations_df.head()

,id,Name,Company,Street,House_Number,Postalcode,City,Lat,Long,State
0,1,star Tankstelle,STAR,Listlandstrasse,16,25992,List,55.015890,8.429530,SH
1,2,Esso Tankstelle,ESSO,TRIFT 2,,25980,WESTERLAND,54.907468,8.308741,SH
2,3,"SYLT, TRIFT",Shell,TRIFT 7,NaN,25980,SYLT,54.904400,8.310700,SH
3,4,DEXX-Station 188,DEXX,Dorfstr.,19,25927,Aventoft,54.901300,8.822810,SH
4,5,team Tankautomat Hesbüll,Raiffeisen,Raiffeisenstr.,1,25927,Hesbüll,54.900900,8.810690,SH


In [7]:
gas_stations_df.to_csv(os.path.join('..', 'data', 'processed', 'Tankstellen_states.csv'))